In [5]:
#Imports and data imports
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn import preprocessing

#Using UCI Machine Learning Repository's database.
#Using data from the " Bike Sharing Dataset Data Set" (https://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset)
#Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science. 

rawData = pd.read_csv(
    "./hour.csv",
    header=[0],
    sep=",")

#Pop columns with unhelpful data
for x in ['instant','dteday','cnt']:
    rawData.pop(x) 
    
#OneHotEncoding for Non-Binary Discrete Variables
def oneHotEncode(df,features):
    one_hot = pd.get_dummies(df[features],prefix=features,columns=features)
    df = df.drop(features,axis=1)
    df = df.join(one_hot)
    return df
rawData = oneHotEncode(rawData,['season','mnth','hr','weekday','weathersit'])

#normalize outputs
features = rawData[['casual','registered']]
min_max_scaler = preprocessing.MinMaxScaler()
rawData[['casual','registered']] = min_max_scaler.fit_transform(features)

#Split data
length = len(rawData)
y_cols = ['casual','registered']
x_cols = list(rawData.columns)
for y in y_cols:
    x_cols.remove(y)


train, test = np.split(rawData.sample(frac=1),[int(length*0.75)])

train_x, train_y = train[x_cols], train[y_cols]
test_x, test_y = test[x_cols], test[y_cols]

In [ ]:
#Most naive approach. No overfitting countermeasures.

EPOCHS = 50
LR = 0.01

def build_model():
    model = keras.Sequential([
        layers.Flatten(input_shape=test_x.shape[1:]),
        layers.Dense(58, activation='relu'),
        layers.Dense(58, activation='relu'),
        layers.Dense(2)
    ])

    optimizer = keras.optimizers.Adam(learning_rate=LR,decay=LR/EPOCHS)
    loss = tf.keras.losses.MeanSquaredError()
    
    model.compile(loss=loss,
                optimizer=optimizer,
                metrics=['mae','mse'])
    return model

model = build_model()
model.fit(train_x, train_y,epochs=EPOCHS)
model.evaluate(test_x, test_y,verbose=2)

In [ ]:
#Dropout implemented
#Notably, the naive approach is better in terms of metrics, but overfitting is evidenced there.

EPOCHS = 50
LR = 0.01

def build_model():
    model = keras.Sequential([
        layers.Flatten(input_shape=test_x.shape[1:]),
        layers.Dropout(0.2),
        layers.Dense(58, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(58, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(2)
    ])

    optimizer = keras.optimizers.Adam(learning_rate=LR,decay=LR/EPOCHS)
    loss = tf.keras.losses.MeanSquaredError()
    
    model.compile(loss=loss,
                optimizer=optimizer,
                metrics=['mae','mse'])
    return model

model = build_model()
model.fit(train_x, train_y,epochs=EPOCHS)
model.evaluate(test_x, test_y,verbose=2)

In [ ]:
#Dropout, checkpointing, and validating implemented
EPOCHS = 50
LR = 0.01

def build_model():
    model = keras.Sequential([
        layers.Flatten(input_shape=test_x.shape[1:]),
        layers.Dropout(0.2),
        layers.Dense(58, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(58, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(2)
    ])

    optimizer = keras.optimizers.Adam(learning_rate=LR,decay=LR/EPOCHS)
    loss = tf.keras.losses.MeanSquaredError()
    
    model.compile(loss=loss,
                optimizer=optimizer,
                metrics=['mae','mse'])
    return model

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="./validatedBikes/checkpoint_{epoch}.ckpt",
        save_best_only=True,
        monitor="val_loss",
        verbose=1,
    )
]

model = build_model()
model.fit(
    train_x, 
    train_y,
    callbacks=callbacks,
    validation_split=0.25,
    epochs=EPOCHS)
model.evaluate(test_x, test_y,verbose=2)

In [ ]:
#Dropout, checkpointing, validating, and regularization implemented
#Metrics are worse than without regularization.
from tensorflow.keras import regularizers

EPOCHS = 50
LR = 0.01

def build_model():
    model = keras.Sequential([
        layers.Flatten(input_shape=test_x.shape[1:]),
        layers.Dropout(0.2),
        layers.Dense(58, 
                activation='relu',
                kernel_regularizer=regularizers.l2(0.0001)),
        layers.Dropout(0.2),
        layers.Dense(58, 
                activation='relu',
                kernel_regularizer=regularizers.l2(0.0001)),
        layers.Dropout(0.2),
        layers.Dense(2)
    ])

    optimizer = keras.optimizers.Adam(learning_rate=LR,decay=LR/EPOCHS)
    loss = tf.keras.losses.MeanSquaredError()
    
    model.compile(loss=loss,
                optimizer=optimizer,
                metrics=['mae','mse'])
    return model

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="./bikes/checkpoint_{epoch}.ckpt",
        save_best_only=True,
        monitor="val_loss",
        verbose=0,
    )
]

model = build_model()
model.fit(
    train_x, 
    train_y,
    callbacks=callbacks,
    validation_split=0.25,
    epochs=EPOCHS)
model.evaluate(test_x, test_y,verbose=2)

In [6]:
#Customizable tensorflow trial.
from tensorflow import keras
from tensorflow.keras import regularizers, layers

def trial(epochs=50,learning_rate=0.01,dropout_val=0.2,l2_val=0.0001,layer_size=58,trialName=None):
    if trialName == None:
        trialName = "Ep{}_Lr{}_Do{}_L2{}_Ls{}".format(epochs,learning_rate,dropout_val,l2_val,layer_size).replace('0.','')
 
    def build_model():
        model = keras.Sequential([
            layers.Flatten(input_shape=test_x.shape[1:]),
            layers.Dropout(dropout_val),
            layers.Dense(layer_size, 
                    activation='relu',
                    kernel_regularizer=regularizers.l2(l2_val)),
            layers.Dropout(dropout_val),
            layers.Dense(layer_size, 
                    activation='relu',
                    kernel_regularizer=regularizers.l2(l2_val)),
            layers.Dropout(dropout_val),
            layers.Dense(2)
        ])

        optimizer = keras.optimizers.Adam(learning_rate=learning_rate,decay=learning_rate/epochs)
        loss = tf.keras.losses.MeanSquaredError()

        model.compile(loss=loss,
                    optimizer=optimizer,
                    metrics=['mae','mse'])
        return model

    callbacks = [
        keras.callbacks.ModelCheckpoint(
            filepath="./"+trialName+"/checkpoint_{epoch}.ckpt",
            save_best_only=True,
            monitor="val_loss",
            verbose=0,
        )
    ]

    model = build_model()
    history = model.fit(
        train_x, 
        train_y,
        callbacks=callbacks,
        validation_split=0.25,
        epochs=epochs,
        verbose=0)
    test_loss, test_mae, test_mse = model.evaluate(test_x, test_y,verbose=0)
    
    print()
    
    #Return a row of data.
    return trialName, \
        epochs, \
        learning_rate, \
        dropout_val, \
        l2_val, \
        layer_size, \
        history.history['loss'][-1], \
        history.history['mae'][-1], \
        history.history['mse'][-1], \
        history.history['val_loss'][-1], \
        history.history['val_mae'][-1], \
        history.history['val_mse'][-1], \
        test_loss, \
        test_mae, \
        test_mse

data = []

#Play around with many variables.
for eps in [50,100,250]:
    for lrs in [0.05,0.01,0.005]:
        for do in [0.2,0.3,0.4,0.5]:
            for l2 in [0.001,0.0001,0.00001]:
                for ls in [29,58,87]:
                    data.append(trial(eps,lrs,do,l2,ls))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do2_L2001_Ls29/checkpoint_1.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do2_L2001_Ls29/checkpoint_2.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do2_L2001_Ls29/checkpoint_4.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do2_L2001_Ls29/checkpoint_5.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do2_L2001_Ls29/checkpoint_6.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do2_L2001_Ls29/checkpoint_12.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do2_L2001_Ls29/checkpoint_16.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do2_L2001_Ls29/checkpoint_18.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do2_L2001_Ls29/checkpoint_36.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do2_L2001_Ls29/checkpoint_49.ckpt/assets

INFO:tensorflow:Assets written to: ./Ep50_L

INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do3_L2001_Ls29/checkpoint_3.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do3_L2001_Ls29/checkpoint_6.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do3_L2001_Ls29/checkpoint_13.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do3_L2001_Ls29/checkpoint_18.ckpt/assets

INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do3_L2001_Ls58/checkpoint_1.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do3_L2001_Ls58/checkpoint_2.ckpt/assets

INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do3_L2001_Ls87/checkpoint_1.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do3_L2001_Ls87/checkpoint_2.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do3_L2001_Ls87/checkpoint_3.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do3_L2001_Ls87/checkpoint_6.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do3_L2001_Ls87/checkpoint_15.ckpt/assets
INFO:tensorflow:Assets written to: ./E

INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do4_L2001_Ls87/checkpoint_4.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do4_L2001_Ls87/checkpoint_5.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do4_L2001_Ls87/checkpoint_7.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do4_L2001_Ls87/checkpoint_10.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do4_L2001_Ls87/checkpoint_24.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do4_L2001_Ls87/checkpoint_28.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do4_L2001_Ls87/checkpoint_31.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do4_L2001_Ls87/checkpoint_39.ckpt/assets

INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do4_L20001_Ls29/checkpoint_1.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do4_L20001_Ls29/checkpoint_2.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do4_L20001_Ls29/checkpoint_3.ckpt/assets
INFO:tensorflow:Assets written to:

INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do5_L2001_Ls87/checkpoint_2.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do5_L2001_Ls87/checkpoint_3.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do5_L2001_Ls87/checkpoint_4.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do5_L2001_Ls87/checkpoint_9.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do5_L2001_Ls87/checkpoint_29.ckpt/assets

INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do5_L20001_Ls29/checkpoint_1.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do5_L20001_Ls29/checkpoint_2.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do5_L20001_Ls29/checkpoint_3.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do5_L20001_Ls29/checkpoint_4.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do5_L20001_Ls29/checkpoint_7.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr05_Do5_L20001_Ls29/checkpoint_9.ckpt/assets
INFO:tensorflow:Assets written to: 

INFO:tensorflow:Assets written to: ./Ep50_Lr01_Do2_L2001_Ls29/checkpoint_15.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr01_Do2_L2001_Ls29/checkpoint_21.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr01_Do2_L2001_Ls29/checkpoint_27.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr01_Do2_L2001_Ls29/checkpoint_28.ckpt/assets

INFO:tensorflow:Assets written to: ./Ep50_Lr01_Do2_L2001_Ls58/checkpoint_1.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr01_Do2_L2001_Ls58/checkpoint_2.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr01_Do2_L2001_Ls58/checkpoint_3.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr01_Do2_L2001_Ls58/checkpoint_10.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr01_Do2_L2001_Ls58/checkpoint_18.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr01_Do2_L2001_Ls58/checkpoint_23.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep50_Lr01_Do2_L2001_Ls58/checkpoint_25.ckpt/assets
INFO:tensorflow:Assets written to:

...

INFO:tensorflow:Assets written to: ./Ep250_Lr01_Do5_L21e-05_Ls87/checkpoint_174.ckpt/assets

INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do2_L2001_Ls29/checkpoint_1.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do2_L2001_Ls29/checkpoint_2.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do2_L2001_Ls29/checkpoint_4.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do2_L2001_Ls29/checkpoint_7.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do2_L2001_Ls29/checkpoint_8.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do2_L2001_Ls29/checkpoint_17.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do2_L2001_Ls29/checkpoint_30.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do2_L2001_Ls29/checkpoint_75.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do2_L2001_Ls29/checkpoint_135.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do2_L2001_Ls29/checkpoint_164.ckpt/assets
INFO:tenso

INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do2_L21e-05_Ls29/checkpoint_166.ckpt/assets

INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do2_L21e-05_Ls58/checkpoint_1.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do2_L21e-05_Ls58/checkpoint_2.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do2_L21e-05_Ls58/checkpoint_9.ckpt/assets

INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do2_L21e-05_Ls87/checkpoint_1.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do2_L21e-05_Ls87/checkpoint_2.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do2_L21e-05_Ls87/checkpoint_4.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do2_L21e-05_Ls87/checkpoint_7.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do2_L21e-05_Ls87/checkpoint_8.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do2_L21e-05_Ls87/checkpoint_106.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do2_L21e-05_Ls87/checkpoint_134.ckp

INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do4_L2001_Ls29/checkpoint_4.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do4_L2001_Ls29/checkpoint_7.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do4_L2001_Ls29/checkpoint_8.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do4_L2001_Ls29/checkpoint_21.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do4_L2001_Ls29/checkpoint_32.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do4_L2001_Ls29/checkpoint_73.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do4_L2001_Ls29/checkpoint_76.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do4_L2001_Ls29/checkpoint_83.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do4_L2001_Ls29/checkpoint_91.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do4_L2001_Ls29/checkpoint_166.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do4_L2001_Ls29/checkpoint_197.ckpt/assets
INFO:tensor

INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do5_L2001_Ls29/checkpoint_2.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do5_L2001_Ls29/checkpoint_3.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do5_L2001_Ls29/checkpoint_11.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do5_L2001_Ls29/checkpoint_12.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do5_L2001_Ls29/checkpoint_14.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do5_L2001_Ls29/checkpoint_20.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do5_L2001_Ls29/checkpoint_29.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do5_L2001_Ls29/checkpoint_43.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do5_L2001_Ls29/checkpoint_54.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do5_L2001_Ls29/checkpoint_89.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do5_L2001_Ls29/checkpoint_116.ckpt/assets
INFO:tensor

INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do5_L21e-05_Ls29/checkpoint_40.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do5_L21e-05_Ls29/checkpoint_81.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do5_L21e-05_Ls29/checkpoint_188.ckpt/assets

INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do5_L21e-05_Ls58/checkpoint_1.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do5_L21e-05_Ls58/checkpoint_2.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do5_L21e-05_Ls58/checkpoint_3.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do5_L21e-05_Ls58/checkpoint_4.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do5_L21e-05_Ls58/checkpoint_5.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do5_L21e-05_Ls58/checkpoint_6.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do5_L21e-05_Ls58/checkpoint_9.ckpt/assets
INFO:tensorflow:Assets written to: ./Ep250_Lr005_Do5_L21e-05_Ls58/checkpoint_10.ckpt/

In [7]:
columns = [
    "trial_name",
    "epochs",
    "learning_rate",
    "dropout_val",
    "l2_val",
    "layer_size",
    "training_loss","training_mae","training_mse",
    "validation_loss","validation_mae","validation_mse",
    "test_loss","test_mae","test_mse"]
df = pd.DataFrame(data,columns=columns)
df.to_csv('trial_metrics.csv')

df

,trial_name,epochs,learning_rate,dropout_val,l2_val,layer_size,training_loss,training_mae,training_mse,validation_loss,validation_mae,validation_mse,test_loss,test_mae,test_mse
0,Ep50_Lr05_Do2_L2001_Ls29,50,0.050,0.2,0.00100,29,0.019345,0.096489,0.018922,0.016509,0.088613,0.016105,0.016498,0.088199,0.016094
1,Ep50_Lr05_Do2_L2001_Ls58,50,0.050,0.2,0.00100,58,0.018531,0.093503,0.018064,0.014453,0.081192,0.013982,0.014379,0.080801,0.013909
2,Ep50_Lr05_Do2_L2001_Ls87,50,0.050,0.2,0.00100,87,0.018290,0.091461,0.017696,0.015366,0.084124,0.014769,0.015486,0.084245,0.014889
3,Ep50_Lr05_Do2_L20001_Ls29,50,0.050,0.2,0.00010,29,0.013259,0.075259,0.012754,0.008215,0.061273,0.007713,0.008224,0.061191,0.007722
4,Ep50_Lr05_Do2_L20001_Ls58,50,0.050,0.2,0.00010,58,0.018826,0.095707,0.018575,0.015051,0.082525,0.014801,0.014955,0.081953,0.014706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,Ep250_Lr005_Do5_L20001_Ls58,250,0.005,0.5,0.00010,58,0.017191,0.086338,0.016168,0.014734,0.076242,0.013715,0.014898,0.076646,0.013879
320,Ep250_Lr005_Do5_L20001_Ls87,250,0.005,0.5,0.00010,87,0.017471,0.087245,0.016427,0.014700,0.078616,0.013659,0.014823,0.078817,0.013783
321,Ep250_Lr005_Do5_L21e-05_Ls29,250,0.005,0.5,0.00001,29,0.016968,0.087836,0.016565,0.014196,0.076671,0.013796,0.014161,0.076614,0.013760
322,Ep250_Lr005_Do5_L21e-05_Ls58,250,0.005,0.5,0.00001,58,0.016070,0.084538,0.015579,0.015089,0.076805,0.014599,0.015112,0.076826,0.014622
